# Learn to Rank

## Import 

In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 

In [2]:
df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/FE_by_postoal_code_without_review.pkl')

## DF Construction
* add relevance

In [3]:
df.columns

Index(['business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date',
       'name', 'address', 'city', 'state', 'postal_code', 'stars_y',
       'review_count', 'is_open', 'attributes', 'categories', 'hours',
       'density', 'entropy', 'competitiveness'],
      dtype='object')

In [4]:
df = df.drop(columns=[ 'business_id', 'stars_x', 'useful', 'funny', 'cool', 'text',
       'date','address','is_open', 'stars_y','attributes','hours'])

In [5]:
# sort by name and review count 
df = df.sort_values(['name', 'review_count'],
              ascending = [True, False]).reset_index(drop=True)

In [6]:
df

,name,city,state,postal_code,review_count,categories,density,entropy,competitiveness
0,1933 Lounge,Indianapolis,IN,46225,160,"Bars, Nightlife, Restaurants, American (Tradit...",6,1.791759,-0.166667
1,1933 Lounge,Fishers,IN,46037,150,"Steakhouses, Nightlife, Bars, Wine Bars, Resta...",6,1.791759,-0.166667
2,4 Rivers Smokehouse,Tampa,FL,33618,572,"Barbeque, Smokehouse, Restaurants, Food, South...",13,2.564949,-0.076923
3,4 Rivers Smokehouse,Tampa Bay,FL,33609,343,"Barbeque, Smokehouse, Food, Event Planning & S...",11,2.397895,-0.090909
4,54th Street Grill & Bar,St. Louis County,MO,63126,146,"American (Traditional), Bars, Nightlife, Burge...",3,1.098612,-0.333333
...,...,...,...,...,...,...,...,...,...
1456,Zydeco's,Belle Chasse,LA,70037,161,"Restaurants, Cajun/Creole, American (New), Sea...",1,-0.0,-1.0
1457,sweetgreen,Philadelphia,PA,19103,213,"Vegan, Soup, Vegetarian, Salad, Restaurants",19,2.944439,-0.052632
1458,sweetgreen,King Of Prussia,PA,19406,91,"Restaurants, Salad, Vegetarian",19,2.944439,-0.052632
1459,uBreakiFix,New Orleans,LA,70130,105,"Local Services, Electronics Repair, IT Service...",18,3.211524,-0.055556


In [17]:
df.loc[df['postal_code']=='46037']['density'].iloc[0]

6

In [23]:
# postal code and feature dict
postal_code_feature_dict = {}

for postal in Counter(df.postal_code):
    postal_code_feature_dict[postal]={}
    postal_code_feature_dict[postal]['density'] = df.loc[df['postal_code']==postal]['density'].iloc[0]
    postal_code_feature_dict[postal]['entropy'] = df.loc[df['postal_code']==postal]['entropy'].iloc[0]
    postal_code_feature_dict[postal]['competitiveness'] = df.loc[df['postal_code']==postal]['competitiveness'].iloc[0]

In [26]:
len(postal_code_feature_dict)

339

In [24]:
len(Counter(df.name))

566

In [25]:
# create relevance score
cnt = 0
for i in Counter(df.name):
    tmp = df[df.name==i].reset_index(drop=True)
    tmp['relevance']=''
    score = 6
    for j in range(len(tmp)):
        tmp['relevance'][j]=score
        score -=1
    if cnt ==0 :
        new_df = tmp
        cnt+=1
    new_df = pd.concat([new_df,tmp])

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# # breakdown affinity & complementary
# from operator import itemgetter
# indices = range(len(df['affinity'][0]))
# a_df = df['affinity'].transform({f'affinity_{i+1}': itemgetter(i) for i in indices})

# indices = range(len(df['complementary'][0]))
# c_df = df['complementary'].transform({f'complementary_{i+1}': itemgetter(i) for i in indices})

In [ ]:
# new_df = new_df.join(a_df)
# new_df = new_df.join(c_df)

## Model

### Create Dataset

In [27]:
new_df.columns

Index(['name', 'city', 'state', 'postal_code', 'review_count', 'categories',
       'density', 'entropy', 'competitiveness', 'relevance'],
      dtype='object')

In [29]:
# split train test 
cnt = 0

for i in Counter(df.name):
    tmp = new_df[new_df.name==i]
    
    if cnt ==0:
        
        train_df = tmp.iloc[:-1,:]
        test_df = tmp.iloc[-1:,:]
        cnt+=1
    else:
        train_df = pd.concat([train_df ,tmp.iloc[:-1,:] ])
        test_df = pd.concat([test_df ,tmp.iloc[-1:,:] ])
    


In [30]:
print(train_df.shape , test_df.shape)

(897, 10) (566, 10)


In [40]:
pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})

,postal_code,density,entropy,competitiveness
0,46225,6.0,1.791759,-0.166667
1,46037,6.0,1.791759,-0.166667
2,33618,13.0,2.564949,-0.076923
3,33609,11.0,2.397895,-0.090909
4,63126,3.0,1.098612,-0.333333
...,...,...,...,...
334,18969,1.0,-0.000000,-1.000000
335,46235,1.0,-0.000000,-1.000000
336,08055,1.0,-0.000000,-1.000000
337,18915,1.0,-0.000000,-1.000000


In [34]:
test_df

,name,city,state,postal_code,review_count,categories,density,entropy,competitiveness,relevance
1,1933 Lounge,Fishers,IN,46037,150,"Steakhouses, Nightlife, Bars, Wine Bars, Resta...",6,1.791759,-0.166667,5
1,4 Rivers Smokehouse,Tampa Bay,FL,33609,343,"Barbeque, Smokehouse, Food, Event Planning & S...",11,2.397895,-0.090909,5
1,54th Street Grill & Bar,Shiloh,IL,62269,142,"American (New), Restaurants, Bars, Sandwiches,...",4,1.386294,-0.25,5
1,55 South,Brentwood,TN,37027,160,"Bars, Nightlife, Comfort Food, Southern, Resta...",10,2.302585,-0.1,5
1,7venth Sun Brewery,Tampa,FL,33604,101,"Breweries, Pubs, Beer Bar, Nightlife, Bars, Food",3,1.098612,-0.333333,5
...,...,...,...,...,...,...,...,...,...,...
1,Zinburger,Tucson,AZ,85715,636,"Burgers, Venues & Event Spaces, American (New)...",4,1.386294,-0.25,5
4,Zoes Kitchen,Metairie,LA,70006,76,"Restaurants, Salad, Kebab, Sandwiches, Mediter...",4,1.386294,-0.25,2
1,Zydeco's,Belle Chasse,LA,70037,161,"Restaurants, Cajun/Creole, American (New), Sea...",1,-0.0,-1.0,5
1,sweetgreen,King Of Prussia,PA,19406,91,"Restaurants, Salad, Vegetarian",19,2.944439,-0.052632,5


In [43]:
# reconstruct testing set

new_test = pd.DataFrame()

for i in Counter(df.name):
    subset = pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})
    subset['relevance']=0
    subset[~df['postal_code'].isin(train_df[train_df.name==i].postal_code)]
    subset[~df['postal_code'].isin(test_df[test_df.name==i].postal_code)]
    subset['name']=i
    subset = pd.concat([subset , test_df[['name','density', 'entropy', 'competitiveness','relevance']]])
    new_test = pd.concat([new_test , subset])

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [44]:
print(train_df.shape , new_test.shape)

(897, 10) (512230, 6)


In [46]:

for i in ['density', 'entropy', 'competitiveness','relevance']:
    train_df[i] = train_df[i].astype('float')
    new_test[i] = new_test[i].astype('float')

### Training

In [47]:
train_df.columns

Index(['name', 'city', 'state', 'postal_code', 'review_count', 'categories',
       'density', 'entropy', 'competitiveness', 'relevance'],
      dtype='object')

In [48]:
from lightgbm import LGBMRanker
train_features = ['density', 'entropy', 'competitiveness']
model = LGBMRanker(objective="lambdarank")

In [49]:
get_group_size = lambda df: df.reset_index().groupby("name")['name'].count()

train_groups = get_group_size(train_df).to_numpy()
test_groups = get_group_size(new_test).to_numpy()

print(sum(train_groups) , sum(test_groups))

897 512230


In [58]:
model.fit(
    train_df[train_features],
    train_df[['relevance']],
    group=train_groups,
    eval_set=[(new_test[train_features],new_test[['relevance']])],
    eval_group=[test_groups],
    eval_at=10,
    eval_metric=['ndcg'])

[1]	valid_0's ndcg@10: 0.487044
[2]	valid_0's ndcg@10: 0.487044
[3]	valid_0's ndcg@10: 0.280257
[4]	valid_0's ndcg@10: 0.52064
[5]	valid_0's ndcg@10: 0.573805
[6]	valid_0's ndcg@10: 0.52064
[7]	valid_0's ndcg@10: 0.52064
[8]	valid_0's ndcg@10: 0.532131
[9]	valid_0's ndcg@10: 0.532131
[10]	valid_0's ndcg@10: 0.506176
[11]	valid_0's ndcg@10: 0.506176
[12]	valid_0's ndcg@10: 0.506176
[13]	valid_0's ndcg@10: 0.532131
[14]	valid_0's ndcg@10: 0.506176
[15]	valid_0's ndcg@10: 0.506176
[16]	valid_0's ndcg@10: 0.506176
[17]	valid_0's ndcg@10: 0.506176
[18]	valid_0's ndcg@10: 0.506176
[19]	valid_0's ndcg@10: 0.506176
[20]	valid_0's ndcg@10: 0.506176
[21]	valid_0's ndcg@10: 0.506176
[22]	valid_0's ndcg@10: 0.506176
[23]	valid_0's ndcg@10: 0.506176
[24]	valid_0's ndcg@10: 0.506176
[25]	valid_0's ndcg@10: 0.506176
[26]	valid_0's ndcg@10: 0.506176
[27]	valid_0's ndcg@10: 0.506176
[28]	valid_0's ndcg@10: 0.506176
[29]	valid_0's ndcg@10: 0.506176
[30]	valid_0's ndcg@10: 0.506176
[31]	valid_0's ndcg@10

LGBMRanker(objective='lambdarank')

In [ ]:
len([(test_df[train_features],test_df['relevance'])])

In [ ]:
[(test_df[train_features],test_df['relevance'])]